# ADM - HW 3

In [2]:
import functions 

from bs4 import BeautifulSoup
import requests
import os 
import time
import pandas as pd 
import csv 
import nltk
import calendar
from nltk.corpus import stopwords
import numpy as np
from collections import Counter
from functools import reduce
import pickle 
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
import heapq
from collections import Counter
import re


# 1. Data Collection

## 1.1 Get the list of master's degree courses

The code for this section can be found in the *links_retriever.py* file. 


Our process for retrieving the lisks consist in first creating a list containing the urls to all the 6000 courses contained in the 400 pages, then, after making sure we have all the urls by checking the lenght of this list, we write them in a *.txt. file. The main reason we decided not to write directly the urls on a file is because we wanted to be able to check quickly if we had all the urls, since this process gave us some troubles.

To compile the list:
1. we designed a function, *extract_masters* that for each page in the catalogue, retrieves all the 15 links to the courses and returns them in a list;
2. we iterate on the firsts 400 pages and call the above mentioned function on every page, updating the list with all the links by appending the resuls obtained for each age to the "main list".

To iterate on the 400 pages, we noticed that the url of th various pages is *https://www.findamasters.com/masters-degrees/msc-degrees/?PG=* followed by the number of the page in the MSc courses catalogue we're in, so we updated the url of each page by changing the number of the page at each iteration.




## 1.2 Crawl master's degree pages 

The code for this section can be found in *crawler.py*. 




## 1.3 Parse downloaded pages

First of all, we need to load the dataset from the *.csv* file we created before.

In [3]:
msc_degrees = pd.read_csv('msc_degrees.csv')

To see if we created the DataFrame in the correct way, run the following line. 

In [4]:
msc_degrees

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration,url
0,Advanced Physiotherapy Practice - MSc,Glasgow Caledonian University,School of Health and Life Sciences,Full time,Progress your career as a physiotherapist wit...,"January, September",Please see the university website for further...,MSc,1 Year Full Time / 2-3 Years Part Time,Glasgow,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...
1,Advanced Master in Innovation & Strategic Mana...,Solvay Brussels School,Economics and Management,Full time,Programme overview The Advanced Master in I...,September,18.000 €,MA MSc Other Pre-Masters Masters Module,10 months,Brussels,Belgium,On Campus,https://www.findamasters.com/masters-degrees/...
2,"Accounting, Financial Management and Digital B...",University of Reading,Henley Business School,Full time,Embark on a professional accounting career wi...,September,Please see the university website for further...,MSc,1 year full time,Reading,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...
3,Analytical Toxicology MSc,King’s College London,Faculty of Life Sciences & Medicine,Full time,The Analytical Toxicology MSc is a unique stu...,See Course,Please see the university website for further...,MSc,Full-time: One year,London,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...
4,Accounting and Finance - MSc,University of Leeds,Leeds University Business School,Full time,Businesses and governments rely on sound fina...,September,"UK: £18,000 (Total) International: £34,750 (...",MSc,1 year full time,Leeds,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,"Masters of Science in Business, Supply Chain A...",Oregon State University,School of Business,Full time,Master of Science in Business (MSB) Our Mas...,See Course,Please see the university website for further...,MSc,12 months,Corvallis,USA,Online,https://www.findamasters.com/masters-degrees/...
5996,Material Culture & Artefact Studies - MSc/PgDip,University of Glasgow,College of Arts & Humanities,Full time,Material culture and artefact studies combine...,September,Please see the university website for further...,MSc PGDip,"9-12 months full-time, 18-24 months part-time",Glasgow,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...
5997,Master's of Computer Science,Harbour.Space University,Masters Programmes,Full time,Harbour.Space’s Master's of Computer Science ...,"September, January","€29,900/year",MSc,1 year or 2 years,Barcelona,Spain,On Campus,https://www.findamasters.com/masters-degrees/...
5998,Master's of Financial Technology (Fintech),Harbour.Space University,Masters Programmes,Full time,Harbour.Space's FinTech Master programme ...,"September, January","€29,900/year",MBA MSc,1 Year,Barcelona,Spain,On Campus,https://www.findamasters.com/masters-degrees/...


# 2. Search Engine

## 2.0 Preprocessing

### 2.0.0  Preprocessing the text

We are required to preprocess all the informations retrieved for the courses, but since loosing punctuation in the url will lead us to loose its functionality, we won't preprocess that column (also, the info contained in the url aren't textual info). Moreover, we won't preprocess the fees column since there's a particoular preprocessing required for that column.

- 'description': removing punctuation and stopwords, stemming
- 'startDate': eliminating everithing that is not a month     (oss: calendar.month_name contains an empty name as first element)


Before being able to proceed, we had to download 'punkt' and 'stopwords' from nltk. 

In [5]:
#'descritpion' column 
stopw = stopwords.words('english')
msc_degrees['description_clean'] = msc_degrees.description.apply(lambda row: nltk.word_tokenize(row)).apply(lambda row: ([nltk.PorterStemmer().stem(word) for word in row if (word.isalnum() and (not word in stopw))])) 

#'startDate' column
msc_degrees['startDate_clean']= msc_degrees.startDate.apply(lambda row: nltk.word_tokenize(row)).apply(lambda row: ' '.join([month for month in row if month in calendar.month_name[1:]]))

### 2.0.1 - Preprocessing the fees columns

First of all, we notice that there are a lot of courses for which we are addressed to the University's site to see fees' informations, and for these courses, th 'fees' columns says: *"Please see the university website for further information on fees for this course."*. For this reason, we replace such stirng with an empty one under the *'fees_EUR'* column in order to speed up operations.

We notice that we now have 4009 empty lines for the fees values, which makes our operation actually useful.  
Now let's get started eith the rest of the pre-proessing of the fees column.

problem : between the ones withouth matches, we have a lot of rows with structure like 
"UK Fees: 2022/23 fees TBC*;2021/22 fees - 10400   International Fees: 2022/23 fees TBC*;2021/22 fees - 17900".


For these lines, the country is always UK, as we suspected, so we assume that the currency is £, even if not explicitly written. 
For these reasons, for these lines we procede in the following way:
1. match 'UK FEES' at the beginnning of the line
2. search for all the numeric values, clean them by removing '.' and ','
3. save numerical values + £ in the 'fees_clean' column 


In order to convert all the retrieved fees value into numeric values representing values with the same currency, we need to convert all the currency symbols and abbreviations found in the dataset into their corresponding ISO currency. To do so, we created a dictionary  that has as keys the currency found in the dataset and as values their ISO correspondacy. This dictionary is loaded here as 'ISO_currency_dict' and has previously been saved into "ISOcurrency.pkl" file. Its creation can be seen in file "ISOcurrency_file_creation".

In [6]:
# Load ISO currency converter dictionary
with open('ISOcurrency.pkl', 'rb') as iso_file:
    ISO_currency_dict = pickle.load(iso_file)

iso_file.close()

In [7]:
# 'fees_EUR' column init: same as fees, but without the "Please see the university website for further information on fees for this course." lines
msc_degrees['fees_EUR'] = msc_degrees.fees.apply(lambda row: row.strip().replace('Please see the university website for further information on fees for this course.', ''))

#looking for fees numeric values 
''' defining regex patterns to look for:
   pattern1: currency, numeric value
   pattern2: numeric value, currency
   pattern_UKfees: matches its content at the beginning of the string (see explaination above in markdown)'''

pattern1 = re.compile(u'([$¢£¤¥֏؋৲৳৻૱௹฿៛\u20a0-\u20bd\uff04\uffe0\uffe1\uffe5\uffe6]|'r'USD|EUR|GBP|JPY|INR|AUD|CAD|HK|euro|euros|US$|ISK|RMB|SEK|Euro|Euros|CHF|Eur)'r'( {0,1}[0-9]+[.,]{0,1}[0-9]+)+')
pattern2 = re.compile(r'([0-9]+[.,]{0,1}[0-9]+)+ {0,1}'u'([$¢£¤¥֏؋৲৳৻૱௹฿៛\u20a0-\u20bd\uff04\uffe0\uffe1\uffe5\uffe6]|'r'USD|EUR|GBP|JPY|INR|AUD|CAD|HK|euro|euros|US$|ISK|RMB|SEK|Euro|Euros|CHF|Eur)')
pattern_UKfees = r'^UK Fees:'

# retrieving fees values 
msc_degrees['fees_EUR'] = msc_degrees.fees_EUR.apply(lambda row: re.findall(pattern1, row) + [el[::-1] for el in re.findall(pattern2, row)] +     #all matches for pattern1 & pattern 2 in a list of tuples + reversing pattern2 matches
                                                         [ ('£', numeric_value.strip()) for numeric_value in re.findall(r'[ *]{1}[0-9]+ ', row + ' ') if re.search(pattern_UKfees, row.lstrip()) ] #'UK Fees:' lines: matching and retrieving needed numeric values + storing them in appropriate way
                                                         ).apply(lambda row: [(fees[0], fees[1].replace('.', '').replace(',','').replace('*','')) for fees in row])   #cleaning numeric values

#getting change_rates to convert the fees
change_rates = functions.get_changerates('EUR', ISO_currency_dict)

# converting all the  & returning max
msc_degrees['fees_EUR'] = msc_degrees.fees_EUR.apply(lambda row: [functions.currency_converter(el, 'EUR', ISO_currency_dict, change_rates) for el in row] #fees values in euros
                                                       ).apply(functions.get_max_currency) # getting max value for each row; if it's empty, we'll have a nan 


After all the cleaning operations, the dataset now looks like this: 

In [8]:
msc_degrees.head(5)

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration,url,description_clean,startDate_clean,fees_EUR
0,Advanced Physiotherapy Practice - MSc,Glasgow Caledonian University,School of Health and Life Sciences,Full time,Progress your career as a physiotherapist wit...,"January, September",Please see the university website for further...,MSc,1 Year Full Time / 2-3 Years Part Time,Glasgow,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...,"[progress, career, physiotherapist, within, nh...",January September,NaN
1,Advanced Master in Innovation & Strategic Mana...,Solvay Brussels School,Economics and Management,Full time,Programme overview The Advanced Master in I...,September,18.000 €,MA MSc Other Pre-Masters Masters Module,10 months,Brussels,Belgium,On Campus,https://www.findamasters.com/masters-degrees/...,"[programm, overview, the, advanc, master, inno...",September,18000.0
2,"Accounting, Financial Management and Digital B...",University of Reading,Henley Business School,Full time,Embark on a professional accounting career wi...,September,Please see the university website for further...,MSc,1 year full time,Reading,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...,"[embark, profession, account, career, academ, ...",September,NaN
3,Analytical Toxicology MSc,King’s College London,Faculty of Life Sciences & Medicine,Full time,The Analytical Toxicology MSc is a unique stu...,See Course,Please see the university website for further...,MSc,Full-time: One year,London,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...,"[the, analyt, toxicolog, msc, uniqu, studi, co...",,NaN
4,Accounting and Finance - MSc,University of Leeds,Leeds University Business School,Full time,Businesses and governments rely on sound fina...,September,"UK: £18,000 (Total) International: £34,750 (...",MSc,1 year full time,Leeds,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/...,"[busi, govern, reli, sound, financi, knowledg,...",September,34750.0


## 2.1 Conjunctive query 

### 2.1.1 Create your index!

#### Creating *vocabulary* file.     
To do so, we first create a list, *vocabulary_list*, containing all the words in the '*description_clean*' column of the dataset, then we set each word's *term_id* as its implicit index in this list. To do so, zip the previous list with another explicitly containg the indeces we choose.
''', converted in string type (this way we don't have any problem when writing the *vocabulary.tsv* file).'''

In [9]:
# set to True to re-write the vocabulary file
if False:
    # creating vocabulary lists as a tool to write the vocabulary file 
    vocabulary_list = sorted(list(Counter(reduce(lambda x, y: x+y, msc_degrees.description_clean.values)).keys()))
    vocabulary_idx = dict(zip(vocabulary_list, list(range( len(vocabulary_list)))))
    
    # save dictionary to vocabulary.pkl file
    with open('vocabulary.pkl', 'wb') as v_file:
        pickle.dump(vocabulary_idx, v_file)

    v_file.close()

#### Inverted index creation

In [10]:
# set to True to re-write the inverted index file
if False: 
    # Read dictionary pkl file
    with open('vocabulary.pkl', 'rb') as v_file:
        vocabulary_dict = pickle.load(v_file)

    #function to get the documents in which term appears
    get_documents_list = lambda word:  list(msc_degrees.loc[msc_degrees.description_clean.apply(lambda row: word in row )].index)

    #initializing inverted_index_dict
    inverted_index = dict()

    #updating the inverted_index dictionary
    for key in vocabulary_dict:
        #saving needed values in order to call the funcion only once per term 
        doc_list =  get_documents_list(key) 

        #updating dict
        inverted_index.update({vocabulary_dict[key] : doc_list})

    # save dictionary to inverted_index.pkl file
    with open('inverted_index.pkl', 'wb') as invidx_file:
        pickle.dump(inverted_index, invidx_file)

    invidx_file.close()


### 2.1.2 Executing the query

First of all, we need to load the files for the query execution

In [11]:
# Read dictionary pkl file
with open('vocabulary.pkl', 'rb') as v_file:
    vocabulary_dict = pickle.load(v_file)

v_file.close()

#Read inverted_index pkl file
with open('inverted_index.pkl', 'rb') as invidx_file:
    inverted_index = pickle.load(invidx_file)

invidx_file.close()

- Now we have to preprocess the query in the same way we preprocessed the 'description' column to get a match: We define a function that does that, then we apply it to the query
- then we get the index of the documents_ids in which the words appear 
- return list of dovìcuments_ids

In [12]:
#getting the query from input and stripping it
query = input('Type to search:').strip()

#preprocessing the query
query = functions.preprocess_query(query)

#searching documents
doc_sat_query = functions.get_documents_conjunctive_query(query, vocabulary_dict, inverted_index)

#Results visualizatioin
if doc_sat_query == []:
    print("This query didn't produce any result")
else: 
    print('Results:')

results_columns = ['courseName', 'universityName', 'description', 'url']
msc_degrees.loc[doc_sat_query, results_columns]

Results:


,courseName,universityName,description,url
4097,Global Meetings and Events Management MSc / PGDip,University College Birmingham,Become part of an events industry worth an es...,https://www.findamasters.com/masters-degrees/...
3,Analytical Toxicology MSc,King’s College London,The Analytical Toxicology MSc is a unique stu...,https://www.findamasters.com/masters-degrees/...
4,Accounting and Finance - MSc,University of Leeds,Businesses and governments rely on sound fina...,https://www.findamasters.com/masters-degrees/...
2055,Clinical Research - MSc,Cardiff University,"Why study this course This part-time, dist...",https://www.findamasters.com/masters-degrees/...
7,Addictions MSc,King’s College London,Join us for an online session for prospective...,https://www.findamasters.com/masters-degrees/...
...,...,...,...,...
1998,Clinical Geriatrics - MSc,Cardiff University,Why study this course The MSc Clinical Ger...,https://www.findamasters.com/masters-degrees/...
2020,Clinical Neuropsychology - MSc,University of Bristol,Professional programmes in Clinical Neuropsyc...,https://www.findamasters.com/masters-degrees/...
2028,Clinical Ophthalmic Practice MSc,University College London,Register your interest in graduate study at U...,https://www.findamasters.com/masters-degrees/...
2046,Clinical Pharmacy - MSc,University of Sunderland,The Clinical Pharmacy MSc has been designed t...,https://www.findamasters.com/masters-degrees/...


Checking if we have the TAs' restults

In [13]:
res = msc_degrees.loc[doc_sat_query, results_columns]
names = ['Analytical Toxicology MSc', 'Addictions MSc', 'Accounting and Finance - MSc', 'Allergy - MSc/PGDip/PGCert']
res[res['courseName'].isin(names)]

,courseName,universityName,description,url
3,Analytical Toxicology MSc,King’s College London,The Analytical Toxicology MSc is a unique stu...,https://www.findamasters.com/masters-degrees/...
4,Accounting and Finance - MSc,University of Leeds,Businesses and governments rely on sound fina...,https://www.findamasters.com/masters-degrees/...
7,Addictions MSc,King’s College London,Join us for an online session for prospective...,https://www.findamasters.com/masters-degrees/...
527,Accounting and Finance - MSc,University of Sussex,On this MSc you’ll advance your accounting an...,https://www.findamasters.com/masters-degrees/...
1011,Allergy - MSc/PGDip/PGCert,Imperial College London,Allergy is an increasing global health proble...,https://www.findamasters.com/masters-degrees/...


## 2.2 Conjunctive query & Ranking score

### 2.2.1 Inverted index - second version

To create the inverted_index_tfidf dictionary, we observe that the ids we gave to the terms in the vocabulary are the indeces in the previous vocabulary list sorted by alphabetial order. When computing the tfidf score, by looking at a dataframe computed by the matrix, by changing the names of the columns, we saw that the terms wre stored in alhabetical order, leading us to be able to use the tfidf matrix indeces to retrieve the tfidf score for every couple term-document we were interested in. + we used the old inverted_index dictionary. 

In [14]:
if False:
    # calculating tfidf 
    tfidf = TfidfVectorizer(lowercase=False, tokenizer=lambda text: text)  #input='content', lowercase=False, tokenizer=lambda text: text
    tfidf_matrix = tfidf.fit_transform(msc_degrees.description_clean)

    #Read inverted_index pkl file (needed to construct inverted_index with tfidf)
    with open('inverted_index.pkl', 'rb') as invidx_file:
        inverted_index = pickle.load(invidx_file)

    invidx_file.close()

    #creating inverted index (with tfidf) dictionary
    inverted_index_tfidf = dict()
    for term_id in inverted_index:
        inverted_index_tfidf.update( { term_id: dict([ (doc_id, tfidf_matrix[(doc_id , term_id)] ) for doc_id in inverted_index[term_id] ]) } )

    #saving the dictionary as a pickle file
    with open('inverted_index_tfidf.pkl', 'wb') as tfidf_file:
            pickle.dump(inverted_index_tfidf, tfidf_file)

    tfidf_file.close()

### 2.2.2 Execute the query

in order to get the top-k documents regarding the cosine similarity with the query, we decided to: 
1. preprocessing the query
2. getting all the documents containing the query (note that if a document doesn't contain the query, their cosine similarity in the tfidf representation is zero);
3. compute the cosine similarity of these retrived documents with the query, than storing them in a heap structure
4. return the top-k documents as a list
5. show results in a dataframe 

let us observe that we should take into account the different lengths of documents, which throw off the tfidf even with the same number of occurencies of a word (numerical values differing only slightly should be treated in the same way)

In [15]:
#Loading tfidf inverted index pkl file
with open('inverted_index_tfidf.pkl', 'rb') as invidx_file:
    inverted_index_tfidf = pickle.load(invidx_file)

invidx_file.close()

In [16]:
#getting and preprocessing query 
query = input('Type to search: ')
query = functions.preprocess_query(query)

#get search results
topkresults_idx_cossim = functions.search_engine_tfidf(query, inverted_index_tfidf, vocabulary_dict )

#showing results
if topkresults_idx_cossim == []:
    print("There are no results for this query")
else: 
    print('Results:')
    results_columns = ['courseName', 'universityName', 'description', 'url']
    res = msc_degrees.loc[[el[0] for el in topkresults_idx_cossim], results_columns ]
    res['cossim'] = [el[1][0] for el in topkresults_idx_cossim]

res

Results:


,courseName,universityName,description,url,cossim
2291,Computing,University of East London,Do you want to be at the forefront of solving...,https://www.findamasters.com/masters-degrees/...,0.996022
941,Advancing Practice Sensory Integration (MSc),Sheffield Hallam University,Develop your knowledge and understanding of t...,https://www.findamasters.com/masters-degrees/...,0.996022
2255,Computer Science MSc,University of East London,Do you want to be at the forefront of solving...,https://www.findamasters.com/masters-degrees/...,0.996022
236,MSc - Economics,Durham University,Our MSc programmes in Economics will give you...,https://www.findamasters.com/masters-degrees/...,0.993725
2361,Construction Management - MSc,Xi’an Jiaotong-Liverpool University,"In fast-growing countries around the world, t...",https://www.findamasters.com/masters-degrees/...,0.993725
2364,Construction Management (part time) - MSc,Xi’an Jiaotong-Liverpool University,"In fast-growing countries around the world, t...",https://www.findamasters.com/masters-degrees/...,0.993725
2436,Countering Extremist Crime and Terrorism MSc,University College London,Register your interest in graduate study at U...,https://www.findamasters.com/masters-degrees/...,0.993725
515,Accounting - MSc,Bangor University,This degree programme provides the opportunit...,https://www.findamasters.com/masters-degrees/...,0.993725
4805,International Financial Management MSc,University of Groningen,How do you manage international financial act...,https://www.findamasters.com/masters-degrees/...,0.993725
4900,International Master of Science in Fire Safety...,University of Edinburgh,Programme description The International Ma...,https://www.findamasters.com/masters-degrees/...,0.993725
